##  Installing Libraries and Packages

In [ ]:
!pip install detoxify

In [ ]:
from detoxify import Detoxify
from tqdm import tqdm
import torch
import os
import pandas as pd
import math

## Function to Import Data files in a Folder

In [ ]:
def import_data(import_folder:str):
    """Walks through the import folder and yields a list of dicts for each file.
    Will only process json, csv, xlsx and txt files.

    Args:
        import_folder (str, optional): The folder holding the data. Defaults to "Import".

    Yields:
        [list]: list of dictionaries, representing each row from the file
    """
    for _, _, fnames in os.walk(import_folder):
        with tqdm(total=len([x for x in fnames if '.json' in x or '.xlsx' in x or '.csv' in x])) as pbar:
            for fname in fnames:
                pbar.set_description(fname.replace('.json',''))
                if '.json' in fname:
                    df = pd.read_json(f"{import_folder}/{fname}")
                    data = df.T.to_dict().values()
                    yield data, fname
                elif '.xlsx' in fname:
                    df = pd.read_excel(f"{import_folder}/{fname}", engine='openpyxl')
                    data = df.T.to_dict().values()
                    del df
                    yield data, fname
                elif '.csv' in fname:
                    df = pd.read_csv(f"{import_folder}/{fname}")
                    data = df.T.to_dict().values()
                    del df
                    yield data, fname
                elif '.txt' in fname:
                    print("Text FileName is ", f"{import_folder}/{fname}")
                    df = pd.read_csv(f"{import_folder}/{fname}", names=['text'], sep="\n", header=None)
                    data = df.T.to_dict().values()
                    del df
                    # print("Dictionary  ", data)
                    yield data, fname
                else:
                    print("Not supported Format ", f"{import_folder}/{fname}")
                    pass
                pbar.update(1)

## Function to Clean the Text Column - with `@` and `.com`

In [ ]:
def clean_text(text:str):
    if type(text) == float:
        return None
    if text and ''.join(text.split()):
        if type(text) == bytes: #Decoding byte strings
            text = text.decode('utf-8')
        #Removing emails + ***.com urls
        text = ' '.join([item for item in text.split() if '@' not in item and '.com' not in item])
        text = ' '.join(text.split()) #removing all multiple spaces
        if text: return text
    return None


## Function to Break the List of Data with Items per Chunk

In [ ]:
def chunk_lst(lst:list, items_per_chunk:int):
    """Breaks a list into chunks
    Args:
        lst ([list]): List to chunk
        items_per_chunk ([int]): Number of items per list
    Yields:
        [list]: a chunk of lst, with size 'items_per_chunk'
    """
    for i in range(0, len(lst), items_per_chunk):
        yield lst[i:i + items_per_chunk]


## Function to Calculate Detoxicity of Text

In [ ]:
from detoxify import Detoxify
results = Detoxify('unbiased').predict(['Pass it on.','Please do not skip any regulatory steps or licensing requirements'])
print("results Dictionary ", results)
print("results only Toxicity ", results.get('toxicity'))

results Dictionary  {'toxicity': [0.012605313211679459, 0.00037768384208902717], 'severe_toxicity': [7.25553763913922e-06, 1.300658027503232e-06], 'obscene': [0.0002587903290987015, 2.0923429474350996e-05], 'identity_attack': [0.0004194467328488827, 7.918340998003259e-05], 'insult': [0.0007003389182500541, 0.00010092252341564745], 'threat': [0.003826275235041976, 1.8575134163256735e-05], 'sexual_explicit': [6.774796202080324e-05, 1.2177173630334437e-05]}
results only Toxicity  [0.012605313211679459, 0.00037768384208902717]


In [ ]:
def get_scores(text, model):
    with torch.no_grad():
        print("Text Size - ", len(text))
        if type(text) == str:
            result = model.predict(text)
            return result.get('toxicity')
        elif type(text) == list:
            results = []
            chunk_size = 100
            for l in tqdm(chunk_lst(text, chunk_size), desc="Processing Scores", total=math.ceil(len(text)/chunk_size)):
                un_formated_results = model.predict(l)
                print("un_formated_results Dictionary ", un_formated_results)
                results += un_formated_results.get('toxicity')
        return results


## Function to Process data and write the Results. 

In [ ]:
def process_data(data:list, text_columns:list, model:Detoxify) ->list:
    """Computes the toxicity score for a batch of data.
    The toxicity score will be added into each dictionary as 'text_column + "_toxicity"'

    Args:
        data (list): list of dictionaries
        text_column (list): the id of the text column you want to compute
        model (Detoxify): the detoxify model you wish to use

    Returns:
        list: list of dictionaries
    """
    #Cleaning text
    for text_column in text_columns:
        texts = []
        for row in data:
            cleaned_text = clean_text(row[text_column])
            text =  cleaned_text if cleaned_text else ''
            texts.append(text)
        results = get_scores(texts, model)
        print("results length: ", len(results))
    return results

## Input Parameters for the SimpleToxicity Calcultor 

In [ ]:
model = Detoxify("unbiased", device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")) #device='cude'
process_sentiment = True
#Set output format
select_output_format = 3
output_formats = ['json', 'csv', 'xlsx']
#Text column name
text_columns = ["text"]

## Run the code 
1.   Input - import_folder -  The folder holding the data. 
2.   Output - filename_toxicity.csv 



In [46]:
def process_file_toxicity(folder_path: str):
  for data, fname in import_data(folder_path):
      updated_data = process_data(data, text_columns, model)
      print("List Dictionary ", updated_data, fname)
      df = pd.DataFrame(updated_data)
      # Write toxicity score to csv file appending filename + '_toxicity'
      save_fname = fname.split('.')[0]
      df.to_csv(f"{folder_path}/{save_fname}_toxicity.csv")


process_file_toxicity("sample_data/sample")

sample_text.txt: : 0it [00:00, ?it/s]

Text FileName is  sample_data/sample/sample_text.txt
Text Size -  650



Processing Scores:  14%|█▍        | 1/7 [00:38<03:52, 38.79s/it]

un_formated_results Dictionary  {'toxicity': [0.006176656112074852, 0.09421318024396896, 0.9876813888549805, 0.5534142851829529, 0.0010689786868169904, 0.0006262899842113256, 0.04457143694162369, 0.0019281518179923296, 0.0009448841447010636, 0.003355340799316764, 0.003069218248128891, 0.005304948892444372, 0.9554316997528076, 0.03878629952669144, 0.0016195171047002077, 0.9327911734580994, 0.006107184570282698, 0.005304201040416956, 0.004267391283065081, 0.07490035146474838, 0.021986693143844604, 0.00469698291271925, 0.9801448583602905, 0.7687169313430786, 0.0019492259016260505, 0.07617585361003876, 0.00048348598647862673, 0.000625847838819027, 0.0015811769990250468, 0.0006873845122754574, 0.0007043851655907929, 0.0007474974263459444, 0.0005855009076185524, 0.0011946685845032334, 0.0017894640332087874, 0.0006001638830639422, 0.019388563930988312, 0.6862991452217102, 0.3684613108634949, 0.0012777973897755146, 0.000641746271867305, 0.0007797820726409554, 0.0005577397532761097, 0.891021549


Processing Scores:  29%|██▊       | 2/7 [01:21<03:24, 40.84s/it]

...

un_formated_results Dictionary  {'toxicity': [0.00264122080989182, 0.0017241511959582567, 0.001609350205399096, 0.0006924783228896558, 0.003010596614331007, 0.0006255663465708494, 0.000786977238021791, 0.0032901151571422815, 0.005383170675486326, 0.0007199427927844226, 0.0015725619159638882, 0.08902355283498764, 0.024215789511799812, 0.00732064712792635, 0.008106092922389507, 0.0006573318969458342, 0.0013945158571004868, 0.19483131170272827, 0.0012334877392277122, 0.054241009056568146, 0.002164045814424753, 0.006075454410165548, 0.3527185022830963, 0.036945559084415436, 0.11847197264432907, 0.0005703667993657291, 0.001483110710978508, 0.000790897523984313, 0.0004297071136534214, 0.04432373121380806, 0.0019082104554399848, 0.013834069482982159, 0.000491725339088589, 0.009566977620124817, 0.0007374932174570858, 0.0009596024174243212, 0.0007870803237892687, 0.44576627016067505, 0.06259220838546753, 0.0015287887072190642, 0.006447039544582367, 0.008579326793551445, 0.021411728113889694, 0.


Processing Scores:  86%|████████▌ | 6/7 [04:10<00:42, 42.06s/it]

un_formated_results Dictionary  {'toxicity': [0.001334231928922236, 0.0008730939007364213, 0.020565146580338478, 0.04532483220100403, 0.030708536505699158, 0.001577033195644617, 0.0022877489682286978, 0.002332555828616023, 0.26881206035614014, 0.0022074023727327585, 0.0016210675239562988, 0.0018955591367557645, 0.4145672917366028, 0.06701108068227768, 0.011453869752585888, 0.0012598454486578703, 0.07570558786392212, 0.000810476893093437, 0.0007216408266685903, 0.04786790907382965, 0.00143322569783777, 0.4542599618434906, 0.9790076017379761, 0.007183115929365158, 0.000853683624882251, 0.01259012334048748, 0.015126976184546947, 0.0030668466351926327, 0.0016547981649637222, 0.001494147814810276, 0.01738060638308525, 0.10246945917606354, 0.002776416717097163, 0.021237554028630257, 0.029860222712159157, 0.009068709798157215, 0.013326013460755348, 0.0007603423437103629, 0.975531280040741, 0.002516945358365774, 0.9918158650398254, 0.0013118614442646503, 0.016999734565615654, 0.001130874850787


Processing Scores: 100%|██████████| 7/7 [04:29<00:00, 38.54s/it]
sample_text.txt: : 1it [04:29, 269.92s/it]

un_formated_results Dictionary  {'toxicity': [0.011766381561756134, 0.014106383547186852, 0.1688280552625656, 0.0015720183728262782, 0.004991237074136734, 0.001921459217555821, 0.004265221767127514, 0.007135974708944559, 0.2867707312107086, 0.007375153712928295, 0.17124047875404358, 0.04990343004465103, 0.04751679673790932, 0.0025050987023860216, 0.9743623733520508, 0.003568044863641262, 0.000446880905656144, 0.47795984148979187, 0.014500615186989307, 0.01953447051346302, 0.0022588518913835287, 0.00394114013761282, 0.01094344723969698, 0.0013423266354948282, 0.0034708271268755198, 0.854538083076477, 0.015490734949707985, 0.07870922982692719, 0.00436824094504118, 0.9672456383705139, 0.000808783108368516, 0.06233867257833481, 0.005240981467068195, 0.0029085250571370125, 0.0004279481072444469, 0.0011158745037391782, 0.04163290187716484, 0.001471393508836627, 0.4422164857387543, 0.004973712842911482, 0.0006931840907782316, 0.014394182711839676, 0.1561122089624405, 0.4194512367248535, 0.315

## Save toxicity Values to CSV files

In [ ]:
save_fname = fname.split('.')[0]
df.to_csv(f"{save_fname}_toxicity.csv")

In [ ]:
## cleantext() 
len(updated_data)

650